In [28]:
from cryptography.x509 import ocsp
from cryptography.x509.oid import ExtensionOID
import cryptography
from cryptography import x509
from cryptography.hazmat.backends import default_backend
import base64
import subprocess
from urllib.parse import urlparse,unquote
from cryptography.x509.oid import ExtensionOID
import requests
from bs4 import BeautifulSoup

In [23]:
def get_serial_from_url(url):    
    path=unquote(urlparse(url).path.split('/')[-1])
    # print(path)
    data = base64.b64decode(path)
    x=ocsp.load_der_ocsp_request(data)
    return hex(x.serial_number)[2:].upper()

def get_names_from_cert(cert):
    # Assuming 'cert' is your loaded certificate
    try:
        # Get the SAN extension from the certificate
        san_extension = cert.extensions.get_extension_for_oid(ExtensionOID.SUBJECT_ALTERNATIVE_NAME)
        san_names = san_extension.value

        # Extract the SANs as strings
        san_list = []
        for name in san_names:
            if isinstance(name, cryptography.x509.DNSName):
                san_list.append(name.value)
            elif isinstance(name, cryptography.x509.IPAddress):
                san_list.append(str(name.value))
            # Include additional types as necessary, e.g., EmailAddress, URI, etc.

    except cryptography.x509.ExtensionNotFound:
        # Handle the case where the SAN extension is not present
        san_list = []
    subject=cert.subject.rfc4514_string()
    if subject.startswith("CN="):
        subject=subject[3:].split(",")[0]
    san_list.append(subject)
    return list(set(san_list))
    
def ask_crt_sh(sn:str)->list[x509.Certificate]:
    url=f"https://crt.sh/?serial={sn}&match=%3D&deduplicate=Y"
    list_of_certs=requests.get(url)
    bs=BeautifulSoup(list_of_certs.text)
    try:
        cert_rows=bs.findAll('table')[2].findAll('tr')[1:]
    except IndexError:
        return []
    certs=[]
    for tr in cert_rows:
        cert_id=tr.findAll('td')[0].text
        url=f"https://crt.sh/?d={cert_id}"
        # print(url)
        pem_data=requests.get(url).text.encode('utf8')
        cert = x509.load_pem_x509_certificate(pem_data, default_backend())
        certs.append(cert)
    return certs

In [44]:
get_serial_from_url("http://ocsp.globalsign.com/ca/gstsacasha384g4/MFEwTzBNMEswSTAJBgUrDgMCGgUABBS2enBWzCNkWKN%2FFhoLZmlPnDczoAQU6hbGaefjy1dFOTOk8EC%2B0MO9ZZYCEAFIkD3CirynoRlNDBxXuCk%3D")

'148903DC28ABCA7A1194D0C1C57B829'

In [42]:
out=subprocess.run('certutil -urlcache',shell=True,universal_newlines=True,capture_output=True)
urls=[x for x in out.stdout.split("\n\n") if "/MF" in x and x.startswith('http')]

In [43]:
import traceback
for u in urls:
    try:
        serial=get_serial_from_url(u)
        certs=ask_crt_sh(serial)
        for c in certs:
            print(get_names_from_cert(c),serial,u)
    except Exception as e:
        traceback.print_exc()
        print(u)


# cert=parsex509cn()

https://crt.sh/?d=924467853
['Sectigo RSA Code Signing CA,O=Sectigo Limited,L=Salford,ST=Greater Manchester,C=GB'] 1DA248306F9B2618D082E0967D33D36A http://ocsp.usertrust.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBTNMNJMNDqCqx8FcBWK16EHdimS6QQUU3m%2FWqorSs9UgOHYm8Cd8rIDZssCEB2iSDBvmyYY0ILgln0z02o%3D
https://crt.sh/?d=12151918068
https://crt.sh/?d=12151920132
['cache6-sgp1.steamcontent.com'] 3CE8257ABD19291BD0B6E24559E670265BE http://r3.o.lencr.org/MFMwUTBPME0wSzAJBgUrDgMCGgUABBRI2smg%2ByvTLU%2Fw3mjS9We3NfmzxAQUFC6zF7dYVsuuUAlA5h%2BvnYsUwsYCEgPOgler0ZKRvQtuJFWeZwJlvg%3D%3D
['cache6-sgp1.steamcontent.com'] 3CE8257ABD19291BD0B6E24559E670265BE http://r3.o.lencr.org/MFMwUTBPME0wSzAJBgUrDgMCGgUABBRI2smg%2ByvTLU%2Fw3mjS9We3NfmzxAQUFC6zF7dYVsuuUAlA5h%2BvnYsUwsYCEgPOgler0ZKRvQtuJFWeZwJlvg%3D%3D
https://crt.sh/?d=4267304700
['Sectigo Public Code Signing CA R36,O=Sectigo Limited,C=GB'] 621D6D0C52019E3B9079152089211C0A http://ocsp.sectigo.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBSdE3gf41WAic8Uh9lF92%2BIJqh5qwQUMuu

Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\132772286.py", line 5, in <module>
    certs=ask_crt_sh(serial)
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\2877236656.py", line 5, in ask_crt_sh
    cert_rows=bs.findAll('table')[2].findAll('tr')[1:]
              ~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


http://ocsp.globalsign.com/ca/gstsacasha384g4/MFEwTzBNMEswSTAJBgUrDgMCGgUABBS2enBWzCNkWKN%2FFhoLZmlPnDczoAQU6hbGaefjy1dFOTOk8EC%2B0MO9ZZYCEAFIkD3CirynoRlNDBxXuCk%3D


Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\132772286.py", line 5, in <module>
    certs=ask_crt_sh(serial)
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\2877236656.py", line 5, in ask_crt_sh
    cert_rows=bs.findAll('table')[2].findAll('tr')[1:]
              ~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


http://ocsp.globalsign.com/ca/gstsacasha384g4/MFEwTzBNMEswSTAJBgUrDgMCGgUABBS2enBWzCNkWKN%2FFhoLZmlPnDczoAQU6hbGaefjy1dFOTOk8EC%2B0MO9ZZYCEAFIkD3CirynoRlNDBxXuCk%3D?Content-Type: application/ocsp-request
https://crt.sh/?d=11321029551
['store.steampowered.com', 'login.steampowered.com', 'partner.steampowered.com', 'underlords.com', 'partner.steamgames.com', 'support.steampowered.com', 'help.steampowered.com', 'steamcommunity.com', 'www.valvesoftware.com', 'store.steampowered.com,O=Valve Corp,L=Bellevue,ST=Washington,C=US,2.5.4.5=602 290 773,2.5.4.15=Private Organization,1.3.6.1.4.1.311.60.2.1.2=Washington,1.3.6.1.4.1.311.60.2.1.3=US', 'api.steampowered.com'] 465458657730A631153614A9C8DA2FA http://ocsp.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBRJ9L2KGL92BpjF3kAtaDtxauTmhgQUPdNQpdagre7zSmAKZdMh1Pj41g8CEARlRYZXcwpjEVNhSpyNovo%3D
https://crt.sh/?d=8669060484
['*.github.com', 'github.com', '*.github.com,O=GitHub\\, Inc.,L=San Francisco,ST=California,C=US'] B890537125F4510D6372A1028A929A5 ht

...

http://ocsp.sectigo.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBQ5suEceKjAJbxseAmHFkQ9FrhTWQQUDuE6qFM6MdWKvsG7rWcaA4WtNA4CEBfb2arwu%2Fmyb20Ga%2FXRRWA%3D
https://crt.sh/?d=9633426659
['Microsoft Azure RSA TLS Issuing CA 04,O=Microsoft Corporation,C=US'] 9F96EC295555F24749EAF1E5DCED49D http://ocsp.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBQ50otx%2Fh0Ztl%2Bz8SiPI7wEWVxDlQQUTiJUIBiV5uNu5g%2F6%2BrkS7QYXjzkCEAn5bsKVVV8kdJ6vHl3O1J0%3D
https://crt.sh/?d=4304297828
['DigiCert Global G2 TLS RSA SHA256 2020 CA1,O=DigiCert Inc,C=US'] CF5BD062B5602F47AB8502C23CCF066 http://ocsp.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBQ50otx%2Fh0Ztl%2Bz8SiPI7wEWVxDlQQUTiJUIBiV5uNu5g%2F6%2BrkS7QYXjzkCEAz1vQYrVgL0erhQLCPM8GY%3D?Content-Type: application/ocsp-request
https://crt.sh/?d=9633426800
['Microsoft Azure RSA TLS Issuing CA 03,O=Microsoft Corporation,C=US'] 5196526449A5E3D1A38748F5DCFEBCC http://ocsp.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBQ50otx%2Fh0Ztl%2Bz8SiPI7wEWVxDlQQUTiJUIBiV5uNu5g%2F6%2BrkS7QYXjzkCEAUZZSZEm

Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\132772286.py", line 5, in <module>
    certs=ask_crt_sh(serial)
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\2877236656.py", line 5, in ask_crt_sh
    cert_rows=bs.findAll('table')[2].findAll('tr')[1:]
              ~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


http://ocsp.pki.goog/gts1c3/MFIwUDBOMEwwSjAJBgUrDgMCGgUABBTHLnmK3f9hNLO67UdCuLvGwCQHYwQUinR%2Fr4XN7pXNPZzQ4kYU83E1HScCEQDDidhelLJLaQnsKqlsZYpa
https://crt.sh/?d=11686944976
['pubsub.googleapis.com', 'dataflow.googleapis.com', 'edgecert.googleapis.com', 'monitoring.googleapis.com', 'servicecontrol.googleapis.com', 'bigtable.googleapis.com', 'datastore.googleapis.com', 'android.googleapis.com', 'cloudtrace.googleapis.com', 'fcm.googleapis.com', 'logging.googleapis.com', 'stackdriver.googleapis.com', 'spanner.googleapis.com', 'runtimeconfig.googleapis.com', 'firestore.googleapis.com', 'dlp.googleapis.com', 'clouderrorreporting.googleapis.com', 'container.googleapis.com'] 41558EC630096910ABA2B7362E3B8FD http://ocsp.pki.goog/gts1c3/MFEwTzBNMEswSTAJBgUrDgMCGgUABBTHLnmK3f9hNLO67UdCuLvGwCQHYwQUinR%2Fr4XN7pXNPZzQ4kYU83E1HScCEAQVWOxjAJaRCrorc2LjuP0%3D


Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\132772286.py", line 5, in <module>
    certs=ask_crt_sh(serial)
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_2588\2877236656.py", line 5, in ask_crt_sh
    cert_rows=bs.findAll('table')[2].findAll('tr')[1:]
              ~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


http://ocsp.pki.goog/gts1c3/MFIwUDBOMEwwSjAJBgUrDgMCGgUABBTHLnmK3f9hNLO67UdCuLvGwCQHYwQUinR%2Fr4XN7pXNPZzQ4kYU83E1HScCEQCw14DGj8KB%2FwqJau317vru
https://crt.sh/?d=3232541597
['Microsoft Azure ECC TLS Issuing CA 02,O=Microsoft Corporation,C=US'] E8DBE5EA610E6CBB569C736F6D7004B http://ocsp.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBTrjrydRyt%2BApF3GSPypfHBxR5XtQQUs9tIpPmhxdiuNkHMEWNpYim8S8YCEA6Nvl6mEObLtWnHNvbXAEs%3D
https://crt.sh/?d=3232541594
['Microsoft Azure ECC TLS Issuing CA 05,O=Microsoft Corporation,C=US'] CE59C30FD7A83532E2D0146B332F965 http://ocsp.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBTrjrydRyt%2BApF3GSPypfHBxR5XtQQUs9tIpPmhxdiuNkHMEWNpYim8S8YCEAzlnDD9eoNTLi0BRrMy%2BWU%3D
https://crt.sh/?d=12088997114
https://crt.sh/?d=12088990336
['cache8-sgp1.steamcontent.com'] 4C6984A1405ECFACA4E8D93A4C7BEC2469D http://r3.o.lencr.org/MFMwUTBPME0wSzAJBgUrDgMCGgUABBRI2smg%2ByvTLU%2Fw3mjS9We3NfmzxAQUFC6zF7dYVsuuUAlA5h%2BvnYsUwsYCEgTGmEoUBez6yk6Nk6THvsJGnQ%3D%3D
['cache8-sgp1.steamcontent.

In [17]:
print(get_subject_names(cert))


['1password.com', 'www.1password.com']


In [41]:
for p in ask_crt_sh('4BB81D3BBC027D2B293A0B9EB1A813E6247'):
    print(p)
    print(get_names_from_cert(p))

https://crt.sh/?d=12011453729
https://crt.sh/?d=12011453260
<Certificate(subject=<Name(CN=1password.com)>, ...)>
['1password.com', 'www.1password.com']
<Certificate(subject=<Name(CN=1password.com)>, ...)>
['1password.com', 'www.1password.com']


In [25]:
out=subprocess.run('certutil -urlcache',shell=True,universal_newlines=True,capture_output=True)
urls=[x for x in out.stdout.split("\n\n") if "/MF" in x and x.startswith('http')]